In [ ]:
from blockchain import Blockchain, Block

blockchain = Blockchain()

genesis = blockchain.chain[0]
b = blockchain.generate_new_block()
print(genesis.__dict__, "\n\n")
print(b.__dict__)

# trying out signatures

In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

# waarom heb ik het gevoel dat het veel makkeljker dan dit kan

def makeKeyPair():
    private_key = rsa.generate_private_key(
                    public_exponent=65537,
                    key_size=2048,
                  )

    public_key = private_key.public_key()

    return private_key, public_key

def signSomething(private_key, something):
    signature = private_key.sign(
        something,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    
    return signature

def verifySomething(public_key, signature, something):
    try:
        public_key.verify(
            signature,
            something,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
    except:
        print("Invalid Signature")


In [ ]:
# making the key pairs
privkey, pubkey = makeKeyPair()
privkey2, pubkey2 = makeKeyPair()

# signing a message with the private key
message = b"A message I want to sign"
signature = signSomething(privkey2, message)

# verifying it
verifySomething(pubkey, signature, message)

# trying out transactions

In [7]:
from transaction import Transaction, TxIn, TxOut

COINBASE_AMOUNT = 20

bob_addr = "bob123"
alice_addr = "alice321"

# Coinbase transaction to Bob
coinbase_in = TxIn("", 0, "")
coinbase_out = TxOut(bob_addr, COINBASE_AMOUNT)
coinbase_transaction = Transaction([coinbase_in], [coinbase_out])

# Bob's gift to Alice
bob_in = TxIn(coinbase_transaction.id, 0, "signature")
bob_out1 = TxOut(alice_addr, 10)
bob_out2 = TxOut(bob_addr, 10)
bobs_gift_transaction = Transaction([bob_in], [bob_out1, bob_out2])

bobs_gift_transaction.__dict__

{'txIns': [<transaction.TxIn at 0x7f72c78d2d30>],
 'txOuts': [<transaction.TxOut at 0x7f72c78d2e80>,
 'id': 'ab5a1857616aa848ae341a44c0a3b8c22892c6aee4171177695af5b103ee20b7'}